# Intro

FiftyOne in UV environment & CVAT locally with docker compose.

In [ ]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz
import numpy as np
from pathlib import Path

/Users/antoinelavacquery/code/m2rs/fiftyone-dev/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load dataset & launch session

In [ ]:
# Load
fo.list_datasets()
dataset = fo.load_dataset('mae-recto')
# fo.delete_dataset('mae-recto')

# # Create
# mae_path = Path.home() / 'code/m2rs/data/mae/recto'
# dataset = fo.Dataset.from_images_dir(mae_path, name='mae-recto')
# dataset.persistent = True

In [ ]:
# Launch APP
session = fo.launch_app(dataset, auto=False)
session

# Compute stuff

In [ ]:
# COMPUTING STUFF

# Metadata (quick)
dataset.compute_metadata()

# Uniqueness (5 min)
fob.compute_uniqueness(dataset)

# Similarity (30 min)
# model = foz.load_zoo_model("mobilenet-v2-imagenet-torch") # slow, idk why, maybe no leverage of apple mps
model = foz.load_zoo_model("efficientnet-b0-imagenet-torch")

fob.compute_similarity(
    dataset,
    model=model,
    model_kwargs={'device': 'mps'},
    brain_key="similarity",
    skip_existing=True,
    batch_size=50
)

# # Delete all computing actions on the dataset
# dataset.list_brain_runs()
# dataset.delete_brain_runs()


# CVAT labelling

In [ ]:
# Create field
dataset.add_sample_field(
    field_name="ground_truth",
    ftype=fo.EmbeddedDocumentField,
    embedded_doc_type=fo.Detections
)

In [ ]:
# View for annotations
rand_view = dataset.take(300, seed=42)
# Display view in APP
session.view = rand_view

In [ ]:
# A unique identifier for this run
anno_key = "anno_1"

# Send to CVAT
rand_view.annotate(
    anno_key,
    url="http://localhost:8080",
    username="admin",
    password="root",
    label_field="ground_truth2",
    label_type="instances",
    classes=["picture"],
    # launch_editor=True,
)

## Don't forget !
CVAT: Menu -> Finish the job.

In [ ]:
# Get back from CVAT
rand_view.load_annotations(anno_key, cleanup=True)
# session.view = rand_view

# Segment full images

In [ ]:
def segment_full_image(sample: fo.core.sample.Sample, field: str) -> None:
    bounding_box = [0, 0, 1, 1]
    mask = np.full((sample.metadata.height, sample.metadata.width), True, dtype=bool)
    
    if sample[field]:
        detection = sample[field].detections[0]
        detection.bounding_box = bounding_box
        detection.mask = mask
    else:
        detections = []
        detection = fo.Detection(
            label='picture',
            bounding_box=bounding_box,
            mask=mask
        )
        detections.append(detection)
        sample[field] = fo.Detections(detections=detections)
    
    sample.save()

In [ ]:
# Get images to be segmented from a tag
segment_view = dataset.match_tags('segment_full_image')

# Segment the whole subset
for sample in segment_view:
    segment_full_image(sample, 'ground_truth')